In [37]:
from gurobipy import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations
import matplotlib.pyplot as plt
import os, sys


In [119]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 3
        self.truckNum    = 4
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        self.model = None
      
       

    def readData(self):
       
       
        
        self.customerNum = 5
        
        self.nodeNum = self.customerNum + 1
   
        
        self.cus_can_served_by_drone = [i for i in range(1,6)]
            
        
        
        self.drone_distances = [0, 3.77, 5.98, 6.28, 4.22, 7.63]
        
        self.truck_distances  = [[0.0, 6.6, 10.8, 12.5, 5, 8.6, 0],
                               [6.6, 0, 4.2, 20.6, 8.5, 10.7, 6.6],
                               [13, 4.2, 0, 16.8, 12.7, 6.5, 13],
                               [12.9, 20.4, 16.8, 0, 15.6, 10.5, 12.9],
                               [4, 8.5, 12.7, 15.7, 0, 12.1, 4],
                               [8.6, 10.7, 6.5, 10.6, 12.3, 0, 8.6]]
        
        self.truck_time = [[0, 14, 20, 19, 6, 12, 0],
                           [15, 0, 6, 29, 12, 15, 15],
                           [19, 6, 0, 28, 18, 10, 19],
                           [19, 29, 28, 0, 23,19,19],
                           [8, 14, 20, 24, 0, 19, 8],
                           [12, 16, 10, 19,18, 0, 12]
                             ]
        self.drone_time = [i*(60/40) + 2 for i in self.drone_distances]
        self.drone_distances = self.drone_time
        self.truck_distances = self.truck_time                   
#         for i in range(len(self.dt)):
#             self.truck_distances[i].append(self.truck_distances[i][0])
        
        #Decision variables

        # x_ij if (i->j) in vehicle tour
        # # y_im = 1 if cus i assigned to drone m ()

      
    def addConstrs(self):

            
             #SET
            C = [i for i in range (1, self.customerNum + 1)]
            
            N = [i for i in range(self.customerNum + 2)]
            
            N0 = [i for i in range(self.customerNum + 1)]            
            
            N_plus = [i for i in range (1, self.customerNum + 2)]
            
            R = [i for i in range(self.truckNum)]
            U = [k for k in range(self.droneNum)]
            G = [0] + C
            C_U = data.cus_can_served_by_drone
            
            P = [0,7,30,0,5,2]
#             , [0,6,14,20,3,32], [0,5,8,5,6,37]
# [0,0,45,0,10], [0,7,30,0,5,2]
            q_t = 30
            q_d = 5
            
            self.x = {}
            self.y = {}
            self.u = {}
            
            
            #1
            #completion time
            alpha = self.model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
            self.model.update()
            expr = LinExpr(0)
            expr.addTerms(1.0, alpha)

            self.model.setObjective(expr, GRB.MINIMIZE)
            expr.clear()
            #add vars u[i]
            for i in N_plus:
                self.u[i] = self.model.addVar( vtype = GRB.INTEGER, name = "u%d" %i)
            #2
            expr = LinExpr(0)
            for r in R:
                for i in N0:
                    for j in N_plus:
                        if i != j:

                            self.x[i,j,r] = self.model.addVar(0, 1, vtype = GRB.BINARY,name = "x%d,%d,%d" %(i,j,r))

                            self.model.update()
                                
                            expr.addTerms(self.truck_distances[i][j], self.x[i,j,r])
                        else:
                            self.x[i,i,r] = self.model.addVar(0.0, 1.0, 0.0, GRB.BINARY, "x%d,%d,%d" %(i,j,r))

        #         print(expr)

            self.model.addConstr(alpha >= expr, "truckTime")
          
            
            expr.clear()


            #3
            expr = LinExpr(0)
            
            for k in U:
                
                for i in C:

                    if i in C_U:
                        self.y[k,i] = self.model.addVar(0,1, vtype= GRB.BINARY, name =  "y%d,%d" %(k,i))
    #                     print(i)

                        expr.addTerms(self.drone_distances[i], self.y[k,i])
                    else:
                        self.y[k,i] = self.model.addVar(0, 0, vtype = GRB.BINARY, name = "y%d,%d" %(k,i))
                self.model.update()
    #             print(expr)
            self.model.addConstr(alpha >= expr, "dronetime")
            expr.clear()
              
    ###############################        
            
            
            expr = LinExpr(0)
            for k in U:
                for j in C:
                    if j in C_U:
                        expr.addTerms(1.0, self.y[k,j])
                self.model.addConstr(expr <= 1)
                expr.clear()
            expr.clear()           
   ###############################
        
            for j in C:
                expr1 = LinExpr(0)
                expr2 = LinExpr(0)
                for i in N0:
                    if i != j:
                        for r in R:
                            expr1.addTerms(q_t, self.x[i,j,r])
                for k in U:
                    expr2.addTerms(q_d, self.y[k, j])
                
                self.model.addConstr(expr1 + expr2 >= P[j])
                expr1.clear()
                expr2.clear()
  



            #6
            for r in R:
                for j in C:
                
                    expr1 = LinExpr(0)
                    expr2 = LinExpr(0)

                    for i in N0:
                        if i != j:
                            expr1.addTerms(1.0, self.x[i,j,r])

                    for h in N_plus:
                        if h != j:
                            expr2.addTerms(1.0, self.x[j,h,r])
                    self.model.addConstr(expr1 == expr2, "flow conservation")
                    expr1.clear()
                    expr2.clear()
            expr1.clear()
            expr2.clear()

            #8
            
            for r in R:
                expr = LinExpr(0)
                for j in N_plus:
                    expr.addTerms(1.0, self.x[0,j,r])
                self.model.addConstr(expr <= 1)

                expr.clear()   
            expr.clear()
            #9
            
            
           
            for r in R:
                expr1 = LinExpr(0)
                expr2 = LinExpr(0)
                for i in N0:

                    expr1.addTerms(1.0, self.x[i, self.customerNum + 1, r])
                for j in N_plus:
                    expr2.addTerms(1.0, self.x[0,j,r])
                self.model.addConstr(expr1 == expr2)
                expr1.clear()
                expr2.clear()
            
            
            #9
            for r in R:
                for i in C:
                    for j in N_plus:
                        if i!=j:
                            self.model.addConstr((self.u[i] - self.u[j] + 1) <= (self.customerNum  + 2)*( 1 - self.x[i,j,r]) )
            #10
            for i in N_plus:
                self.model.addConstr(1 <= self.u[i])
                self.model.addConstr(self.u[i] <= self.customerNum + 2)
            

In [120]:
data = Data()
data.model = Model("PDSTSP")
data.readData()

data.truck_distances
data.addConstrs()

data.model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 150 rows, 166 columns and 843 nonzeros
Model fingerprint: 0x7d9d4f5e
Variable types: 1 continuous, 165 integer (159 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 39.0000000
Presolve removed 25 rows and 43 columns
Presolve time: 0.00s
Presolved: 125 rows, 123 columns, 669 nonzeros
Variable types: 0 continuous, 123 integer (117 binary)

Root relaxation: objective 1.772430e+01, 53 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.72430    0    5   39.00000   17.72430  54.6%     -    0s
     0     0   20.61640    0   10   

In [121]:
data.model.printAttr('X')


    Variable            X 
-------------------------
  traveltime           39 
          u1            1 
          u2            2 
          u3            1 
          u4            1 
          u5            1 
          u6            7 
      x0,1,0            1 
      x1,2,0            1 
      x2,6,0            1 
        y0,5            1 
        y1,4            1 
        y2,1            1 


In [112]:
N0 = [i for i in range(data.customerNum + 1)]            
            
N_plus = [i for i in range (1, data.customerNum + 2)]
truck_tours = [(i,j) for i in N0 for j in N_plus if data.x[i,j].x > 0.99 ]
print(truck_tours)
U = [k for k in range(data.droneNum)]

C_U = data.cus_can_served_by_drone

plt.plot(data.i_pot[0], data.i_pot[1], c='r', marker='s')
# plt.scatter(xc[1:], yc[1:], c='b')
for i in range(1, data.customerNum + 1):
    plt.scatter(data.cor_X, data.cor_Y, c='b')
    plt.annotate(str("  ") + str(int(data.cities[i])), (data.cor_X[i], data.cor_Y[i]))
    
for i, j in truck_tours:
    if j != data.customerNum + 1:
        plt.plot([data.cor_X[i], data.cor_X[j]], [data.cor_Y[i], data.cor_Y[j]], c='g', zorder=0)
    else:
        for k, h in truck_tours:
            if h == i:
                plt.plot([data.cor_X[i], data.cor_X[0]], [data.cor_Y[i], data.cor_Y[0]], c='g', zorder=0)
        
    
default_width = 1

for k in U:
    default_width+= 1
    for i in C_U:
        if data.y[k,i].x > 0.99:
            plt.plot([data.i_pot[0],data.cor_X[i]], [data.i_pot[1], data.cor_Y[i]], color='black', linestyle='dashed', linewidth = default_width)

KeyError: (0, 1)